##### <div style="background-color: rgb(230, 230, 230);">

---
# T501-SP24 - Lab Assignment 2
# *Using Spark SQL to Analyze Flight On-Time Data*
---

<div style="background-color: rgb(230, 230, 230);">

---
## Assignment Details

### Objectives
The learning objectives of this assignment are to:
1.	Practice using Spark SQL to query, aggregate, and analyze structured data
2.  Use Spark skills to answer open-ended analysis questions.
2.  Work successfully in a team on a collaborative assignment

### General Instructions
* **Please attempt to do the majority of this assignment with Spark SQL.**  You may, however, find it necessary to mix in commands from the Spark DataFrame API. This is okay.
* Upload the data files accompanying this notebook (**NovOntime.csv**, **Airports.csv**) into your user directory on Ambari in CloudXLab.  
* Upload this Jupyter notebook to your Jupyter directory on CloudXLab. Enter all of your code into this notebook. 
* Your modified Jupyter notebook will serve as your group’s submission for the assignment. You should **also submit a PDF version** of the notebook (through "print" to a PDF or your system, or download as PDF from CloudXLab.)

### Questions to Answer
This assignment contains three types of tasks:
1. Data reading, discovery, and output tasks
2. Specific, closed-ended queries
3. Open-ended analysis questions, usually requiring multiple queries


---
# Part 1 - Data Reading and Discovery Tasks (4 x 5 pts each = 20 pts)

### Task 1.0 - Set up Spark environment
Execute (run) the code in the following cell to initiate the Spark environment and load required libraries.

In [2]:
// This command is necessary for loading the aggregation functions...
import org.apache.spark.sql.functions._  

lastException: Throwable = null


### Task 1.1 - Read Data (5 pts)
For each dataset:
1. Read the .csv file into a Spark dataFrame
1. Then display the first few rows of data.
1. Next, output the number of rows of data in the dataFrame.
1. Finally, create a Spark SQL view from the dataFrame. 

The code to read the airports file has been provided for you; perform a similar set of commands for the OnTime data.

In [3]:
// Path to data set (in HDFS (Ambari))
val directory = "HW 2/" // change this path to fit your setup
val csvFile = directory + "Airports.csv"

// Read and create a temporary view
val airportDF = spark.read.format("csv")
  .option("inferSchema", "true")
  .option("header", "true")
  .load(csvFile)
// verify
airportDF.show(5)
airportDF.printSchema()
println("Number of rows: " + airportDF.count())

// Create a temporary view
airportDF.createOrReplaceTempView("airportView")

+---------+--------------------+-----------+-----+-------+--------+----------+
|IATA_CODE|             AIRPORT|       CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|
+---------+--------------------+-----------+-----+-------+--------+----------+
|      ABE|Lehigh Valley Int...|  Allentown|   PA|    USA|40.65236|  -75.4404|
|      ABI|Abilene Regional ...|    Abilene|   TX|    USA|32.41132|  -99.6819|
|      ABQ|Albuquerque Inter...|Albuquerque|   NM|    USA|35.04022|-106.60919|
|      ABR|Aberdeen Regional...|   Aberdeen|   SD|    USA|45.44906| -98.42183|
|      ABY|Southwest Georgia...|     Albany|   GA|    USA|31.53552| -84.19447|
+---------+--------------------+-----------+-----+-------+--------+----------+
only showing top 5 rows

root
 |-- IATA_CODE: string (nullable = true)
 |-- AIRPORT: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nul

directory = HW 2/
csvFile = HW 2/Airports.csv
airportDF = [IATA_CODE: string, AIRPORT: string ... 5 more fields]


[IATA_CODE: string, AIRPORT: string ... 5 more fields]

### Task 1.2 - Check Databases, Tables, and Table Structure (0 pts)
This code is provided for you. Be sure to update the name of the temp_view as necessary.

In [4]:
// Check the databases, tables, and table contents
spark.catalog.listDatabases().show() // databases
spark.catalog.listTables().show() // tables & views
spark.catalog.listColumns("airportView").show() // table columns 
// do the same for the SQL View containing the NovOntime information

+-------+----------------+--------------------+
|   name|     description|         locationUri|
+-------+----------------+--------------------+
|default|default database|/apps/hive/warehouse|
+-------+----------------+--------------------+

+-----------+--------+-----------+---------+-----------+
|       name|database|description|tableType|isTemporary|
+-----------+--------+-----------+---------+-----------+
|airportview|    null|       null|TEMPORARY|       true|
+-----------+--------+-----------+---------+-----------+

+---------+-----------+--------+--------+-----------+--------+
|     name|description|dataType|nullable|isPartition|isBucket|
+---------+-----------+--------+--------+-----------+--------+
|IATA_CODE|       null|  string|    true|      false|   false|
|  AIRPORT|       null|  string|    true|      false|   false|
|     CITY|       null|  string|    true|      false|   false|
|    STATE|       null|  string|    true|      false|   false|
|  COUNTRY|       null|  string|

### Task 1.3 - Verify Dates in File (5 pts)
From the file name, we might guess (correctly) that the file contains on-time information from the month of November. But for how much of the month, and for what year(s)? 

Write queries to determine the range of values for the Year, Month, Day of Month, and Day of Week contained in the dataset. (Hint: since these values are all numeric, think MIN() and MAX() )

In [5]:

val novOntimeFile = directory + "NovOntime.csv"

// Read and create a temporary view
val novOntimeDF = spark.read.format("csv")
  .option("inferSchema", "true")
  .option("header", "true")
  .load(novOntimeFile)

// Verify the DataFrame
novOntimeDF.show(5)
novOntimeDF.printSchema()

// Create a temporary view
novOntimeDF.createOrReplaceTempView("novOntimeView")


[Stage 8:>                                                          (0 + 2) / 2]+----------+----+-------+-----+----------+---------+-------+------+----+----------+-------+--------+---------------+--------+--------------------+----------+-------+---------+--------+------+----------+-------+--------+---------------+--------+------------------+----------+---------+----------------+--------+--------------+-----------------+-------+--------+-------------+------------+------------+--------+-------------+-----------------+
|IDENTIFIER|Year|Quarter|Month|DayofMonth|DayOfWeek|Carrier|Origin|Dest|CRSDepTime|DepTime|DepDelay|DepDelayMinutes|DepDel15|DepartureDelayGroups|DepTimeBlk|TaxiOut|WheelsOff|WheelsOn|TaxiIn|CRSArrTime|ArrTime|ArrDelay|ArrDelayMinutes|ArrDel15|ArrivalDelayGroups|ArrTimeBlk|Cancelled|CancellationCode|Diverted|CRSElapsedTime|ActualElapsedTime|AirTime|Distance|DistanceGroup|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----------+----+-------+-----+-----

novOntimeFile = HW 2/NovOntime.csv
novOntimeDF = [IDENTIFIER: int, Year: int ... 38 more fields]


[IDENTIFIER: int, Year: int ... 38 more fields]

In [6]:
// find the range of Year, Month, Day of Month, and Day of Week
spark.sql("""
SELECT 
    MIN(Year) AS Min_Year, MAX(Year) AS Max_Year, 
    MIN(Month) AS Min_Month, MAX(Month) AS Max_Month, 
    MIN(DayofMonth) AS Min_DayOfMonth, MAX(DayofMonth) AS Max_DayOfMonth, 
    MIN(DayOfWeek) AS Min_DayOfWeek, MAX(DayOfWeek) AS Max_DayOfWeek 
FROM novOntimeView
""").show()


+--------+--------+---------+---------+--------------+--------------+-------------+-------------+
|Min_Year|Max_Year|Min_Month|Max_Month|Min_DayOfMonth|Max_DayOfMonth|Min_DayOfWeek|Max_DayOfWeek|
+--------+--------+---------+---------+--------------+--------------+-------------+-------------+
|    2010|    2010|       11|       11|             1|            30|            1|            7|
+--------+--------+---------+---------+--------------+--------------+-------------+-------------+



### Task 1.4  - Understand the Data (5 pts)
To get a better sense of the breadth and details of the data, query the database to discover the following basic facts:
* Number and names of ArrTimeBlks
* Number and names of Dest(intation) airports
* Number and names of Origin airports
* Number and names of Carriers

In [7]:
//Number and Names of ArrTimeBlks

spark.sql("""
SELECT 
    COUNT(DISTINCT ArrTimeBlk) AS NumberOfArrTimeBlks, 
    COLLECT_LIST(DISTINCT ArrTimeBlk) AS NamesOfArrTimeBlks
FROM novOntimeView
""").show(false)


[Stage 13:=================================================>    (182 + 8) / 200]+-------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|NumberOfArrTimeBlks|NamesOfArrTimeBlks                                                                                                                                                                                               |
+-------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|19                 |[1300-1359, 1400-1459, 1100-1159, 2100-2159, 2000-2059, 0001-0559, 2200-2259, 0600-0659, 0900-0959, 1800-1859, 1700-1759, 2300-2359, 0700-0759, 1200-1259, 1000-1059, 0800-0859, 1900-1959, 1600-1659, 1500

In [8]:
//Number and Names of Destination Airports (Dest)

spark.sql("""
SELECT 
    COUNT(DISTINCT Dest) AS NumberOfDestAirports, 
    COLLECT_LIST(DISTINCT Dest) AS NamesOfDestAirports
FROM novOntimeView
""").show(false)


+--------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|NumberOfDestAirports|NamesOfDestAirports                                                                                                                                                                                                                                                                                         |
+--------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|60                  |[LGA, 

In [9]:
// Number and Names of Origin Airports (Origin)

spark.sql("""
SELECT 
    COUNT(DISTINCT Origin) AS NumberOfOriginAirports, 
    COLLECT_LIST(DISTINCT Origin) AS NamesOfOriginAirports
FROM novOntimeView
""").show(false)


+----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|NumberOfOriginAirports|NamesOfOriginAirports                                                                                                                                                                                                                                                                                       |
+----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|60                   

In [10]:
//Number and Names of Carriers

spark.sql("""
SELECT 
    COUNT(DISTINCT Carrier) AS NumberOfCarriers, 
    COLLECT_LIST(DISTINCT Carrier) AS NamesOfCarriers
FROM novOntimeView
""").show(false)


[Stage 22:====================================================> (196 + 4) / 200]+----------------+------------------------+
|NumberOfCarriers|NamesOfCarriers         |
+----------------+------------------------+
|6               |[AS, UA, WN, AA, B6, DL]|
+----------------+------------------------+



### Task 1.5 - Write and Compare Parquet File (5 pts)
Write a command to output the file NovOntime.csv into Parquet format. List the sizes of the two files and compute the disk space savings of the parquet version. (Hint 1: consider using the coalesce(1) function to force the data into one file instead of multiple parts. Hint 2: to determine the file sizes, simply go to the Ambari HDFS (Files) interface and refer to the size column.)

* Size of CSV file:
* Size of Parquet file:
* % Savings of Parquet file:

In [11]:
//Write DataFrame to Parquet

novOntimeDF.coalesce(1).write.mode("overwrite").parquet(directory + "NovOntime.parquet")


SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


In [12]:
//Check File Sizes and Compute Savings
//Size of CSV file: 27.5 MB
//Size of Parquet file: 4.9 MB


val sizeOfCsvFile: Double = 27.5 // MB
val sizeOfParquetFile: Double = 4.9 // MB

//calculate the savings percentage
val savings: Double = (sizeOfCsvFile - sizeOfParquetFile) / sizeOfCsvFile
val percentageSavings: Double = savings * 100

println(percentageSavings)



82.18181818181819


sizeOfCsvFile = 27.5
sizeOfParquetFile = 4.9
savings = 0.8218181818181819
percentageSavings = 82.18181818181819


82.18181818181819

##### ---

# Part 2 - Specific Queries (8 x 5 pts each = 40 pts.)

---
### Task 2.1 - Mean Delay by Carrier and Day Of Week
Find the mean arrival delay by carrier and day of week.
    
(Note the difference between ArrDelay and ArrDelayMinutes in the dataset.
We want ArrDelayMinutes for this problem; this is the variable for which flights that arrived early have a delay of 0 rather than a negative value.)


In [13]:


// find the mean arrival delay by carrier and day of week
val meanDelayByCarrierAndDayOfWeek = spark.sql("""
SELECT 
    Carrier, 
    DayOfWeek, 
    AVG(ArrDelayMinutes) AS MeanArrivalDelay
FROM novOntimeView
GROUP BY Carrier, DayOfWeek
ORDER BY Carrier, DayOfWeek
""")

// Show the results
meanDelayByCarrierAndDayOfWeek.show()


[Stage 25:=========>                                                (1 + 5) / 6]+-------+---------+------------------+
|Carrier|DayOfWeek|  MeanArrivalDelay|
+-------+---------+------------------+
|     AA|        1| 9.240923076923076|
|     AA|        2|10.456535141800247|
|     AA|        3| 7.576290852833107|
|     AA|        4|7.5224005001041885|
|     AA|        5| 8.727971614429332|
|     AA|        6| 5.508827908955541|
|     AA|        7| 6.869996113486203|
|     AS|        1|12.388972809667674|
|     AS|        2| 9.362307692307692|
|     AS|        3| 6.745746691871456|
|     AS|        4|4.8410206084396465|
|     AS|        5| 6.545101842870999|
|     AS|        6| 7.022380467955239|
|     AS|        7| 8.171905697445972|
|     B6|        1|14.284249767008388|
|     B6|        2|10.716543209876543|
|     B6|        3| 9.995665634674923|
|     B6|        4|13.315656565656566|
|     B6|        5|10.866514546491729|
|     B6|        6| 9.452365930599369|
+-------+---------+----

meanDelayByCarrierAndDayOfWeek = [Carrier: string, DayOfWeek: int ... 1 more field]


[Carrier: string, DayOfWeek: int ... 1 more field]

---
### Task 2.2 - Mean Delay by Destination and Day of Week
Find mean arrival delay by airport and day of Week for selected airports.
Include only the following airports: IND, ORD, STL, CVG.


In [14]:

// find the mean arrival delay by destination airport and day of week for selected airports
val meanDelayByDestinationAndDayOfWeek = spark.sql("""
SELECT 
    Dest AS Airport, 
    DayOfWeek, 
    AVG(ArrDelayMinutes) AS MeanArrivalDelay
FROM novOntimeView
WHERE Dest IN ('IND', 'ORD', 'STL', 'CVG')
GROUP BY Dest, DayOfWeek
ORDER BY Dest, DayOfWeek
""")

// Show the results
meanDelayByDestinationAndDayOfWeek.show()


+-------+---------+------------------+
|Airport|DayOfWeek|  MeanArrivalDelay|
+-------+---------+------------------+
|    CVG|        1|  8.16891891891892|
|    CVG|        2|   8.9009009009009|
|    CVG|        3|10.552631578947368|
|    CVG|        4| 8.303571428571429|
|    CVG|        5| 6.460869565217391|
|    CVG|        6| 3.830188679245283|
|    CVG|        7|         11.859375|
|    IND|        1|   8.8659793814433|
|    IND|        2| 8.431034482758621|
|    IND|        3| 9.487179487179487|
|    IND|        4| 7.948529411764706|
|    IND|        5| 7.302013422818792|
|    IND|        6| 9.242424242424242|
|    IND|        7|              5.75|
|    ORD|        1|12.939465875370919|
|    ORD|        2| 5.386961722488039|
|    ORD|        3| 7.837600585223116|
|    ORD|        4| 4.538586515028432|
|    ORD|        5| 4.046692607003891|
|    ORD|        6|  4.31453744493392|
+-------+---------+------------------+
only showing top 20 rows



meanDelayByDestinationAndDayOfWeek = [Airport: string, DayOfWeek: int ... 1 more field]


[Airport: string, DayOfWeek: int ... 1 more field]

---
### Task 2.3 - Mean Delay by Day, Time Block, and Destination
Find the mean arrival delay by day of week, arrival time block, and destination.
(Again use ArrDelayMinutes, not ArrDelay.)


In [15]:

// find the mean arrival delay by day of week, arrival time block, and destination
val meanDelayByDayTimeBlockAndDestination = spark.sql("""
SELECT 
    DayOfWeek, 
    ArrTimeBlk, 
    Dest AS Destination, 
    AVG(ArrDelayMinutes) AS MeanArrivalDelay
FROM novOntimeView
GROUP BY DayOfWeek, ArrTimeBlk, Dest
ORDER BY DayOfWeek, ArrTimeBlk, Dest
""")

// Show the results
meanDelayByDayTimeBlockAndDestination.show()


+---------+----------+-----------+------------------+                           
|DayOfWeek|ArrTimeBlk|Destination|  MeanArrivalDelay|
+---------+----------+-----------+------------------+
|        1| 0001-0559|        ABQ|             135.0|
|        1| 0001-0559|        ANC|25.565217391304348|
|        1| 0001-0559|        ATL|               1.1|
|        1| 0001-0559|        AUS|               2.0|
|        1| 0001-0559|        BDL|              12.0|
|        1| 0001-0559|        BNA|              15.0|
|        1| 0001-0559|        BOS| 11.18421052631579|
|        1| 0001-0559|        BUF|10.333333333333334|
|        1| 0001-0559|        BWI| 7.933333333333334|
|        1| 0001-0559|        CMH|               5.0|
|        1| 0001-0559|        DCA|3.2857142857142856|
|        1| 0001-0559|        DFW| 16.27027027027027|
|        1| 0001-0559|        EWR|10.571428571428571|
|        1| 0001-0559|        FLL| 9.944444444444445|
|        1| 0001-0559|        HOU|27.818181818181817|
|

meanDelayByDayTimeBlockAndDestination = [DayOfWeek: int, ArrTimeBlk: string ... 2 more fields]


[DayOfWeek: int, ArrTimeBlk: string ... 2 more fields]

---
### Task 2.4 - Cost of Delays per Carrier
If a minute of delay costs an airline $78, how much, in millions, was lost to <u>carrier, weather, and late aircraft delays?</u> The result should be one total per carrier.

In [16]:
// calculate the cost of delays per carrier
val costOfDelaysPerCarrier = spark.sql("""
SELECT 
    Carrier,
    ROUND(SUM((CarrierDelay + WeatherDelay + LateAircraftDelay) * 78) / 1000000, 2) AS TotalCostInMillions
FROM novOntimeView
GROUP BY Carrier
ORDER BY TotalCostInMillions DESC
""")

// Show the results
costOfDelaysPerCarrier.show()


[Stage 31:=========>                                                (1 + 5) / 6]+-------+-------------------+
|Carrier|TotalCostInMillions|
+-------+-------------------+
|     WN|               39.2|
|     DL|              20.23|
|     AA|              14.51|
|     B6|               7.07|
|     UA|               5.44|
|     AS|               2.57|
+-------+-------------------+



costOfDelaysPerCarrier = [Carrier: string, TotalCostInMillions: double]


[Carrier: string, TotalCostInMillions: double]

---
### Task 2.5 - Delays from Indianapolis for American and Delta
We want to compare departure delays from the Indianapolis airport (IND) for carriers American Airlines (AA) and Delta Airlines (DL). Group the results by Carrier and DayOfWeek.


In [17]:
// compare departure delays for American Airlines (AA) and Delta Airlines (DL) from Indianapolis (IND)
val delaysFromIndianapolis = spark.sql("""
SELECT 
    Carrier, 
    DayOfWeek, 
    AVG(DepDelayMinutes) AS AverageDepartureDelay
FROM novOntimeView
WHERE 
    Origin = 'IND' AND 
    Carrier IN ('AA', 'DL')
GROUP BY Carrier, DayOfWeek
ORDER BY Carrier, DayOfWeek
""")

// Show the results
delaysFromIndianapolis.show()


+-------+---------+---------------------+
|Carrier|DayOfWeek|AverageDepartureDelay|
+-------+---------+---------------------+
|     AA|        1|                  0.6|
|     AA|        2|                 7.24|
|     AA|        3|                  1.4|
|     AA|        4|    4.388888888888889|
|     AA|        5|   12.421052631578947|
|     AA|        6|   1.6666666666666667|
|     AA|        7|                 5.55|
|     DL|        1|    5.538461538461538|
|     DL|        2|             5.765625|
|     DL|        3|    13.78688524590164|
|     DL|        4|    6.627118644067797|
|     DL|        5|                 7.55|
|     DL|        6|    5.084745762711864|
|     DL|        7|   14.044117647058824|
+-------+---------+---------------------+



delaysFromIndianapolis = [Carrier: string, DayOfWeek: int ... 1 more field]


[Carrier: string, DayOfWeek: int ... 1 more field]

---
### Task 2.6 - Delays for American and United at Select Destinations
We'd like to <u>compare American Airlines and United Airlines with flights leaving from St. Louis, Dallas FortWorth, Indianapolis, and JFK.</u> We want to <u>use the average ArriveDelay in minutes</u> as the metric to compare these carriers. We'd like to have an output organized by DayOfWeek followed by Carrier, each showing the average ArrDelayMinutes.


In [18]:
// compare average arrival delay for American Airlines (AA) and United Airlines (UA)
val delaysForAmericanAndUnited = spark.sql("""
SELECT 
    DayOfWeek, 
    Carrier, 
    AVG(ArrDelayMinutes) AS AverageArrivalDelay
FROM novOntimeView
WHERE 
    Origin IN ('STL', 'DFW', 'IND', 'JFK') AND 
    Carrier IN ('AA', 'UA')
GROUP BY DayOfWeek, Carrier
ORDER BY DayOfWeek, Carrier
""")

// Show the results
delaysForAmericanAndUnited.show()


+---------+-------+-------------------+
|DayOfWeek|Carrier|AverageArrivalDelay|
+---------+-------+-------------------+
|        1|     AA|  7.089622641509434|
|        1|     UA| 2.7966101694915255|
|        2|     AA|  13.19537517697027|
|        2|     UA|  4.254237288135593|
|        3|     AA|  7.599645180366647|
|        3|     UA|  2.351063829787234|
|        4|     AA|  7.528883183568678|
|        4|     UA|  1.632183908045977|
|        5|     AA| 10.761356753482739|
|        5|     UA|  2.797752808988764|
|        6|     AA|  5.579268292682927|
|        6|     UA| 4.8933333333333335|
|        7|     AA|  6.166965888689408|
|        7|     UA|  4.586206896551724|
+---------+-------+-------------------+



delaysForAmericanAndUnited = [DayOfWeek: int, Carrier: string ... 1 more field]


[DayOfWeek: int, Carrier: string ... 1 more field]

---
### Task 2.7 - Regional airports by State
We would like to determine the airports located in Indiana and the surrounding states. Find all of the airports listed in the Airports.csv file that are located in the states of Indiana, Illinois, Kentucky, Ohio, and Michigan. Sort the results by State and City.

In [19]:
// find airports in Indiana and surrounding states
val airportsByStateCorrected = spark.sql("""
SELECT 
    AIRPORT AS Name, 
    CITY, 
    STATE
FROM airportView
WHERE 
    STATE IN ('IN', 'IL', 'KY', 'OH', 'MI')
ORDER BY STATE, CITY
""")

// Show the results
airportsByStateCorrected.show()


+--------------------+----------------+-----+
|                Name|            CITY|STATE|
+--------------------+----------------+-----+
|Central Illinois ...|     Bloomington|   IL|
|University of Ill...|Champaign/Urbana|   IL|
|Chicago Midway In...|         Chicago|   IL|
|Chicago O'Hare In...|         Chicago|   IL|
|Quad City Interna...|          Moline|   IL|
|General Wayne A. ...|          Peoria|   IL|
|Abraham Lincoln C...|     Springfield|   IL|
|Evansville Region...|      Evansville|   IN|
|Fort Wayne Intern...|      Fort Wayne|   IN|
|Indianapolis Inte...|    Indianapolis|   IN|
|South Bend Intern...|      South Bend|   IN|
|Cincinnati/Northe...|       Covington|   KY|
|  Blue Grass Airport|       Lexington|   KY|
|Louisville Intern...|      Louisville|   KY|
|Barkley Regional ...|         Paducah|   KY|
|Alpena County Reg...|          Alpena|   MI|
|Detroit Metropoli...|         Detroit|   MI|
|Delta County Airport|        Escanaba|   MI|
|Bishop Internatio...|           F

airportsByStateCorrected = [Name: string, CITY: string ... 1 more field]


[Name: string, CITY: string ... 1 more field]

---
### Task 2.8 - Airports with direct connections to Indianapolis
Print out the City, State, and Airport name of all airports that have a direct connection with Indianapolis. (Yes, this will require a JOIN.) Be sure to list each airport only once.

In [20]:
// find airports with direct connections to Indianapolis
val directConnectionsToIndianapolis = spark.sql("""
SELECT DISTINCT 
    a.CITY, 
    a.STATE, 
    a.AIRPORT AS Name
FROM 
    novOntimeView f
JOIN 
    airportView a
ON 
    f.Dest = a.IATA_CODE OR f.Origin = a.IATA_CODE
WHERE 
    f.Origin = 'IND' OR f.Dest = 'IND'
AND 
    a.IATA_CODE != 'IND'
ORDER BY 
    a.STATE, a.CITY
""")

// Show the results
directConnectionsToIndianapolis.show()


+-----------------+-----+--------------------+                                  
|             CITY|STATE|                Name|
+-----------------+-----+--------------------+
|          Phoenix|   AZ|Phoenix Sky Harbo...|
|      Los Angeles|   CA|Los Angeles Inter...|
|           Denver|   CO|Denver Internatio...|
|          Orlando|   FL|Orlando Internati...|
|            Tampa|   FL|Tampa Internation...|
|          Atlanta|   GA|Hartsfield-Jackso...|
|          Chicago|   IL|Chicago Midway In...|
|     Indianapolis|   IN|Indianapolis Inte...|
|        Baltimore|   MD|Baltimore-Washing...|
|          Detroit|   MI|Detroit Metropoli...|
|      Minneapolis|   MN|Minneapolis-Saint...|
|      Kansas City|   MO|Kansas City Inter...|
|        Las Vegas|   NV|McCarran Internat...|
|Dallas-Fort Worth|   TX|Dallas/Fort Worth...|
|   Salt Lake City|   UT|Salt Lake City In...|
+-----------------+-----+--------------------+



directConnectionsToIndianapolis = [CITY: string, STATE: string ... 1 more field]


[CITY: string, STATE: string ... 1 more field]

---

# Part 3 - Open-Ended Analyses (2 x 10 pts ea = 20 pts.)

---
### Task 3.1 - "Making up time" Policy Analysis
Is there a way the data might tell us <u>which airlines have a strict policy against "making time up in the air"?</u> When there is a Departure delay, you will often hear pilots announce that they are going to "make up time in the air" (fly faster) to get back on schedule.  Some airlines do not like this, because it raises fuel costs.  See if you can tell which airlines discourage this. Explain your analysis and rationale.

In [21]:
// Analyze: "making up time in the air" policy by airlines
val timeMakingPolicyAnalysis = spark.sql("""
SELECT 
    Carrier, 
    AVG(DepDelayMinutes - ArrDelayMinutes) AS AvgTimeMadeUp
FROM novOntimeView
WHERE DepDelayMinutes > 0
GROUP BY Carrier
ORDER BY AvgTimeMadeUp DESC
""")

// Show the results
timeMakingPolicyAnalysis.show()


+-------+-------------------+
|Carrier|      AvgTimeMadeUp|
+-------+-------------------+
|     UA|  6.340970873786408|
|     WN|  4.357851450874707|
|     B6| 2.6141316310807836|
|     AA| 2.4502923976608186|
|     DL| 1.3168550992155053|
|     AS|0.31100478468899523|
+-------+-------------------+



timeMakingPolicyAnalysis = [Carrier: string, AvgTimeMadeUp: double]


[Carrier: string, AvgTimeMadeUp: double]

#### It appears that AS may have a strict policy against "making time up in the air" with an average time made up (ATM) of .311. It is also possible that other airlines such as DL (ATM 1.31) may discourgae it as well within a threshold, however it probably is not a strict policy against "making time up in the air". 

---
### Task 3.2 - Sunday & Monday Delay Analysis

We’d like you to <u>analyze flights that arrived late, specifically examining Sunday and Monday performance for each carrier.</u>  What patterns do you see?  What factors might influence the numbers or analysis?
(For example, if you were to sum the delayed minutes for each carrier, it seems alarming that carrier WN has, by far, the highest total delays.  Do you see any reasons for this?)  Again, focus your analysis on Sunday and Monday flights delayed on arrival at their destination airport, and see what you can analyze to explain what may be happening with these carriers on these particular days. Again, explain your analyses and rationale.

(Hint: Nov 1, 2010 was a Monday.)

In [29]:
// analyze Sunday and Monday flight performance for each carrier, including all delay categories
//Delays are in Total Minutes
val sundayMondayDelays = spark.sql("""
SELECT 
    Carrier, 
    DayOfWeek,
    COUNT(*) AS NumberOfFlights,
    SUM(ArrDelayMinutes) AS ARRDelay,
    SUM(DepDelayMinutes) AS DEPDelay,
    SUM(CarrierDelay) AS CarrierDelay,
    SUM(WeatherDelay) AS WeatherDelay,
    SUM(NASDelay) AS NASDelay, 
    SUM(SecurityDelay) AS SecurityDelay, 
    SUM(LateAircraftDelay) AS LateAircraftDelay
FROM novOntimeView
WHERE DayOfWeek IN (1, 7) AND ArrDelayMinutes > 0
GROUP BY Carrier, DayOfWeek
ORDER BY Carrier, DayOfWeek
""")

// Show the results
sundayMondayDelays.show()



+-------+---------+---------------+--------+--------+------------+------------+--------+-------------+-----------------+
|Carrier|DayOfWeek|NumberOfFlights|ARRDelay|DEPDelay|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+-------+---------+---------------+--------+--------+------------+------------+--------+-------------+-----------------+
|     AA|        1|           2331| 60066.0| 51897.0|     19952.0|      1504.0| 16632.0|          0.0|          13627.0|
|     AA|        7|           1485| 35353.0| 34339.0|     12300.0|       300.0|  8745.0|         12.0|           8771.0|
|     AS|        1|            521| 16403.0| 11613.0|      2478.0|      2137.0|  6405.0|          0.0|           3749.0|
|     AS|        7|            344|  8319.0|  6092.0|      1943.0|        12.0|  2656.0|          0.0|           2415.0|
|     B6|        1|            998| 30654.0| 27672.0|     10948.0|         0.0|  7632.0|         50.0|           9100.0|
|     B6|        7|            6

sundayMondayDelays = [Carrier: string, DayOfWeek: int ... 8 more fields]


[Carrier: string, DayOfWeek: int ... 8 more fields]

In [28]:
// analyze Sunday and Monday flight performance for each carrier, focusing on average delays per flight
val sundayMondayDelaysAverageRounded = spark.sql("""
SELECT 
    Carrier, 
    DayOfWeek,
    COUNT(*) AS NumberOfFlights,
    ROUND(AVG(ArrDelayMinutes), 2) AS ARRDelay,
    ROUND(AVG(DepDelayMinutes), 2) AS DEPDelay,
    ROUND(AVG(CarrierDelay), 2) AS CarrierDelay,
    ROUND(AVG(WeatherDelay), 2) AS WeatherDelay,
    ROUND(AVG(LateAircraftDelay), 2) AS LateAircraftDelay,
    ROUND(AVG(NASDelay), 2) AS NASDelay,
    ROUND(AVG(SecurityDelay), 2) AS SecurityDelay
FROM novOntimeView
WHERE DayOfWeek IN (1, 7) AND ArrDelayMinutes > 0
GROUP BY Carrier, DayOfWeek
ORDER BY Carrier, DayOfWeek
""")

// Show the results
sundayMondayDelaysAverageRounded.show()



+-------+---------+---------------+--------+--------+------------+------------+-----------------+--------+-------------+
|Carrier|DayOfWeek|NumberOfFlights|ARRDelay|DEPDelay|CarrierDelay|WeatherDelay|LateAircraftDelay|NASDelay|SecurityDelay|
+-------+---------+---------------+--------+--------+------------+------------+-----------------+--------+-------------+
|     AA|        1|           2331|   25.77|   22.26|       19.68|        1.48|            13.44|    16.4|          0.0|
|     AA|        7|           1485|   23.81|   23.12|       19.49|        0.48|             13.9|   13.86|         0.02|
|     AS|        1|            521|   31.48|   22.29|       10.28|        8.87|            15.56|   26.58|          0.0|
|     AS|        7|            344|   24.18|   17.71|       12.87|        0.08|            15.99|   17.59|          0.0|
|     B6|        1|            998|   30.72|   27.73|       19.41|         0.0|            16.13|   13.53|         0.09|
|     B6|        7|            6

sundayMondayDelaysAverageRounded = [Carrier: string, DayOfWeek: int ... 8 more fields]


[Carrier: string, DayOfWeek: int ... 8 more fields]

In [94]:
val sundayMondayDelaysAverageRounded = spark.sql("""
SELECT 
    Carrier, 
    DayOfWeek,
    COUNT(*) AS NumberOfFlights,
    ROUND(SUM(COALESCE(ArrDelayMinutes, 0)) / COUNT(*), 2) AS ArrivalDelay,
    ROUND(SUM(COALESCE(CarrierDelay, 0)) / COUNT(*), 2) AS CarrierDelay,
    ROUND(SUM(COALESCE(WeatherDelay, 0)) / COUNT(*), 2) AS WeatherDelay,
    ROUND(SUM(COALESCE(LateAircraftDelay, 0)) / COUNT(*), 2) AS LateAircraftDelay,
    ROUND(SUM(COALESCE(NASDelay, 0)) / COUNT(*), 2) AS NASDelay,
    ROUND(SUM(COALESCE(SecurityDelay, 0)) / COUNT(*), 2) AS SecurityDelay
FROM novOntimeView
WHERE DayOfWeek IN (1, 7) AND ArrDelayMinutes > 0
GROUP BY Carrier, DayOfWeek
ORDER BY Carrier, DayOfWeek
""")

sundayMondayDelaysAverageRounded.show()



+-------+---------+---------------+------------+------------+------------+-----------------+--------+-------------+
|Carrier|DayOfWeek|NumberOfFlights|ArrivalDelay|CarrierDelay|WeatherDelay|LateAircraftDelay|NASDelay|SecurityDelay|
+-------+---------+---------------+------------+------------+------------+-----------------+--------+-------------+
|     AA|        1|           2331|       25.77|        8.56|        0.65|             5.85|    7.14|          0.0|
|     AA|        7|           1485|       23.81|        8.28|         0.2|             5.91|    5.89|         0.01|
|     AS|        1|            521|       31.48|        4.76|         4.1|              7.2|   12.29|          0.0|
|     AS|        7|            344|       24.18|        5.65|        0.03|             7.02|    7.72|          0.0|
|     B6|        1|            998|       30.72|       10.97|         0.0|             9.12|    7.65|         0.05|
|     B6|        7|            653|       29.26|       11.59|        0.0

sundayMondayDelaysAverageRounded = [Carrier: string, DayOfWeek: int ... 7 more fields]


[Carrier: string, DayOfWeek: int ... 7 more fields]

In [31]:
val sundayMondayDelaysAverageRounded = spark.sql("""
SELECT 
    Carrier, 
    DayOfWeek,
    COUNT(*) AS NumberOfFlights,
    ROUND(SUM(COALESCE(ArrDelayMinutes, 0)) / COUNT(*), 2) AS ARRDelay,
    ROUND(SUM(COALESCE(DepDelayMinutes, 0)) / COUNT(*), 2) AS DEPDelay, -- Added average departure delay
    ROUND(SUM(COALESCE(CarrierDelay, 0)) / COUNT(*), 2) AS CarrierDelay,
    ROUND(SUM(COALESCE(WeatherDelay, 0)) / COUNT(*), 2) AS WeatherDelay,
    ROUND(SUM(COALESCE(LateAircraftDelay, 0)) / COUNT(*), 2) AS LateAircraftDelay,
    ROUND(SUM(COALESCE(NASDelay, 0)) / COUNT(*), 2) AS NASDelay,
    ROUND(SUM(COALESCE(SecurityDelay, 0)) / COUNT(*), 2) AS SecurityDelay
FROM novOntimeView
WHERE DayOfWeek IN (1, 7) AND ArrDelayMinutes > 0
GROUP BY Carrier, DayOfWeek
ORDER BY Carrier, DayOfWeek
""")

sundayMondayDelaysAverageRounded.show()


+-------+---------+---------------+--------+--------+------------+------------+-----------------+--------+-------------+
|Carrier|DayOfWeek|NumberOfFlights|ARRDelay|DEPDelay|CarrierDelay|WeatherDelay|LateAircraftDelay|NASDelay|SecurityDelay|
+-------+---------+---------------+--------+--------+------------+------------+-----------------+--------+-------------+
|     AA|        1|           2331|   25.77|   22.26|        8.56|        0.65|             5.85|    7.14|          0.0|
|     AA|        7|           1485|   23.81|   23.12|        8.28|         0.2|             5.91|    5.89|         0.01|
|     AS|        1|            521|   31.48|   22.29|        4.76|         4.1|              7.2|   12.29|          0.0|
|     AS|        7|            344|   24.18|   17.71|        5.65|        0.03|             7.02|    7.72|          0.0|
|     B6|        1|            998|   30.72|   27.73|       10.97|         0.0|             9.12|    7.65|         0.05|
|     B6|        7|            6

sundayMondayDelaysAverageRounded = [Carrier: string, DayOfWeek: int ... 8 more fields]


[Carrier: string, DayOfWeek: int ... 8 more fields]

In [32]:
val sundayMondayDelaysAverageRounded = spark.sql("""
SELECT 
    Carrier, 
    DayOfWeek,
    COUNT(*) AS NumberOfFlights,
    ROUND(SUM(COALESCE(ArrDelayMinutes, 0)) / COUNT(*), 2) AS ARRDelay,
    ROUND(SUM(COALESCE(DepDelayMinutes, 0)) / COUNT(*), 2) AS DEPDelay, -- Added average departure delay
    ROUND(SUM(COALESCE(CarrierDelay, 0)) / COUNT(*), 2) AS CarrierDelay,
    ROUND(SUM(COALESCE(WeatherDelay, 0)) / COUNT(*), 2) AS WeatherDelay,
    ROUND(SUM(COALESCE(LateAircraftDelay, 0)) / COUNT(*), 2) AS LateAircraftDelay,
    ROUND(SUM(COALESCE(NASDelay, 0)) / COUNT(*), 2) AS NASDelay,
    ROUND(SUM(COALESCE(SecurityDelay, 0)) / COUNT(*), 2) AS SecurityDelay
FROM novOntimeView
WHERE DayOfWeek IN (1, 7) AND ArrDelayMinutes > 0
GROUP BY Carrier, DayOfWeek
ORDER BY ARRDelay DESC  -- Adjusted for sorting by highest average arrival delay
""")

sundayMondayDelaysAverageRounded.show()


+-------+---------+---------------+--------+--------+------------+------------+-----------------+--------+-------------+
|Carrier|DayOfWeek|NumberOfFlights|ARRDelay|DEPDelay|CarrierDelay|WeatherDelay|LateAircraftDelay|NASDelay|SecurityDelay|
+-------+---------+---------------+--------+--------+------------+------------+-----------------+--------+-------------+
|     UA|        1|            795|   33.17|   30.57|        6.77|        0.82|            10.83|   11.92|          0.0|
|     AS|        1|            521|   31.48|   22.29|        4.76|         4.1|              7.2|   12.29|          0.0|
|     B6|        1|            998|   30.72|   27.73|       10.97|         0.0|             9.12|    7.65|         0.05|
|     B6|        7|            653|   29.26|   27.03|       11.59|        0.01|             9.06|    5.35|          0.1|
|     DL|        1|           3862|   27.47|   23.34|         7.5|        0.35|             9.08|    7.38|          0.0|
|     DL|        7|           27

sundayMondayDelaysAverageRounded = [Carrier: string, DayOfWeek: int ... 8 more fields]


[Carrier: string, DayOfWeek: int ... 8 more fields]


## Overall Patterns Based on Average Arrival Delays:

#### - Higher Traffic, Lower Delays: It seems that the number of flights may inversely correlate with delays. This could suggest that carriers with more flights have more resources and capacity to manage and mitigate delays.

#### - Day of the Week Differences: The data suggests that Monday is more prone to delays than Sunday. This could be due to a variety of factors, including increased air traffic, the beginning of the work week causing a surge in travel, or operational resets from the weekend.

####  - Carrier-Specific Delays: Certain carriers show a pattern of high delays in specific categories, which may reflect operational challenges unique to each carrier.

## Factors Influencing Delays:

#### 1. Volume of Flights: WN's higher total minutes of delay correlates with its higher volume of flights. More flights can lead to more total delayed minutes even if the average delay per flight is not the highest.

#### 2. Operational Efficiency: The size of the airline and the efficiency of its operations could be a contributing factor. carriers with more flights like WN have complex operations which, when disrupted, can lead to higher late aircraft delays.

#### 3. Carrier-Specific Issues: High carrier delays for B6, DL, and AA might indicate issues related to the airlines' own operations, such as maintenance or crew management.

#### 4. Weather and Environmental Factors: AS's high weather delays on Monday suggest environmental factors that might be specific to the routes they operate on that day or adverse weather conditions.

#### 5. Scheduling Practices: High late aircraft delays, particularly for UA and WN, might indicate tight scheduling and quick turnarounds, which can compound delays.

#### 6. Infrastructure and Traffic Control: High NAS delays for UA and AS on Monday could reflect air traffic control issues, airport congestion, or infrastructure limitations at specific airports.

#### 7. Minimal Security Delays: Since security delays are negligible across carriers, they do not significantly contribute to overall delay patterns.



